# Automatic differentiation with ``autograd`` 


In machine learning, we want models to get better and better as a function of experience. Usually *getting better*, means minimizing a *loss function*, a measure of how *bad* our model is at any time. With neural networks, that loss is usually differentiable, i.e. for each of the model's parameters, we can always determine how much increasing or decreasing it might affect the loss. For complex models, working out these derivatives from scratch can be a pain.

_MXNet_'s autograd package eliminates this tedious work by automatically calculating derivatives for you. Other libraries require you to pre-define and compile symbolic graphs in order to access automatic derivatives. However, ``autograd``, much like the similar package in PyTorch, allows you to take derivatives even when running fully imperative code.

Essentially, every time you make pass through your model, autograd builds a graph on the fly, through which it can immediately backpropagate gradients.

Let's go through it step by step. For this tutorial, we'll only need to import ``mxnet.ndarray``, and ``mxnet.autograd``.

In [92]:
import mxnet as mx
import mxnet.ndarray as nd
from mxnet import autograd

## Attaching Gradients

As a toy example, Let's say that we are interested in differentiating a function ``f = 2 * (x ** 2)`` with respect to parameter x. We can start by assining an initial value of ``x``.

In [71]:
x = mx.nd.array([[1, 2], [3, 4]])

Once we compute the gradient of ``f`` with respect to ``x``, we'll need a place to store it. In _MXNet_, we can tell an NDArray that we plan to store a gradient by invoking its ``atach_grad()`` method.

In [72]:
x.attach_grad()

Now we're going to define out function ``f`` and *MXNet* will generate a computation graph on the fly. It's as if *MXNet* turned on a recording device and captured the exact path by which each variabel was generated. 

Note that building the computation graph requires a nontrivial amount of computation. So we only *MXNet* to build the graph when explicitlt told to do so. We can instruct *MXNet* to start recording by placing code inside a ``with autograd.record():`` block.

In [73]:
with autograd.record():
  y = x * 2
  f = y * x

Let's backprop with f.backward(). When f has more than one entry, f.backward() is equivalent to mx.nd.sum(f).backward().



In [74]:
gradients = nd.array([[1.,1.],[.1,.1]])
f.backward(gradients)

Now, let's see if this is the expected output. Remember that ``y = x * 2``, and ``f = x * y``, so ``f`` should be equal to  ``2 * x * x``.

After, doing backprop with ``f.backward()``, we expect to get back gradient df/dx as follows: dy/dx = ``2``, df/dx = ``4 * x``.

So, if everything went according to plan, ``x.grad`` should consist of an NDArray with the values ``[[4, 8],[12, 16]]``.

In [75]:
print(x.grad)

[[ 4.          8.        ]
 [ 1.20000005  1.60000002]]
<NDArray 2x2 @cpu(0)>


Now that we know the basics, we can do some wild things with autograd, including building diferentiable functions using Pythonic control flow.

In [89]:
a = nd.random_normal(shape=3)
a.attach_grad()

with autograd.record():
    b = a * 2
    while (nd.norm(b) < 1000).asscalar():
        b = b * 2

    if (mx.nd.sum(b) > 0).asscalar():
        c = b
    else :
        c = 100 * b

In [90]:
gradients = nd.array([0.1,1.0,.0001])
c.backward(gradients)

In [91]:
x.grad


[[ 4.          8.        ]
 [ 1.20000005  1.60000002]]
<NDArray 2x2 @cpu(0)>